In [1]:
from utils import create_facts_and_examples
import numpy as np
import pandas as pd
import random, os
np.random.seed(0) #Code B

In [2]:
N = 30
A =  np.bool_(np.random.randint(2, size=N))
B =  np.bool_(np.random.randint(2, size=N))

customer = ['c'+str(i+16000) for i in range(90)]

Fraud = A & B

df = pd.DataFrame({'orig':np.random.choice(customer, size=N), 'destination':np.random.choice(customer, size=N),\
                   'A':A, 'B':B, "Fraud":Fraud})
df.drop(df[(A==0)&(B==0)].index, inplace=True)

df = pd.DataFrame({'orig':np.random.choice(customer, size=N), 'destination':np.random.choice(customer, size=N),\
                   'A':A, 'B':B, "Fraud":Fraud})
df.drop(df[(A==0)&(B==0)].index, inplace=True)

In [3]:
df

,orig,destination,A,B,Fraud
0,c16042,c16051,False,True,False
1,c16077,c16080,True,False,False
2,c16021,c16032,True,True,True
4,c16000,c16000,True,True,True
5,c16010,c16038,True,True,True
6,c16043,c16019,True,False,False
7,c16058,c16046,True,True,True
8,c16023,c16042,True,True,True
9,c16059,c16056,True,False,False
10,c16002,c16060,True,False,False


In [4]:
df.drop(df[df['orig']==df['destination']].index, inplace=True)



In [5]:
fraudsters = df[df['Fraud']==True]

In [6]:
fraudsters[['orig','destination']]

,orig,destination
2,c16021,c16032
5,c16010,c16038
7,c16058,c16046
8,c16023,c16042
13,c16067,c16024
19,c16027,c16019
21,c16041,c16026
22,c16058,c16066
26,c16086,c16014


In [7]:
index_from_merge = pd.merge(df.reset_index(), fraudsters[['destination']].drop_duplicates(), how='inner', left_on='orig', right_on='destination')

In [8]:
index_from_merge

,index,orig,destination_x,A,B,Fraud,destination_y
0,0,c16042,c16051,False,True,False,c16042
1,15,c16046,c16003,False,True,False,c16046


In [9]:
df['Fraud_chain--orig--destination']=False
df.loc[index_from_merge['index'],'Fraud']=True
df.loc[index_from_merge['index'],'Fraud_chain--orig--destination']=True
df['From--orig']=1
df['To--destination']=1

In [10]:
df.loc[index_from_merge['index'],:]

,orig,destination,A,B,Fraud,Fraud_chain--orig--destination,From--orig,To--destination
0,c16042,c16051,False,True,True,True,1,1
15,c16046,c16003,False,True,True,True,1,1


In [11]:
df

,orig,destination,A,B,Fraud,Fraud_chain--orig--destination,From--orig,To--destination
0,c16042,c16051,False,True,True,True,1,1
1,c16077,c16080,True,False,False,False,1,1
2,c16021,c16032,True,True,True,False,1,1
5,c16010,c16038,True,True,True,False,1,1
6,c16043,c16019,True,False,False,False,1,1
7,c16058,c16046,True,True,True,False,1,1
8,c16023,c16042,True,True,True,False,1,1
9,c16059,c16056,True,False,False,False,1,1
10,c16002,c16060,True,False,False,False,1,1
11,c16062,c16077,False,True,False,False,1,1


### creating fact, positive and negative examples

In [12]:
#drop first letter and convert to int
df['orig'] = df['orig'].str[1:].astype('int')
df['destination'] = df['destination'].str[1:].astype('int')

In [13]:
%%script true

create_facts_and_examples(df_= df, target='Fraud', 
                          predicates=['A', 'B', 'From--orig', 'To--destination'], output_dir="examples/chain-fraud-erman")

In [14]:
%%script true

create_facts_and_examples(df_= df, target='Fraud_chain--orig--destination', 
                          predicates=['Fraud', 'From--orig', 'To--destination'], output_dir="examples/transitivity-fraud-rule")

## Create simple transitivity problem

In [15]:
pd.merge(df[['orig','destination']], df[['orig','destination']], \
                            how='inner', left_on=['orig', 'destination'], right_on=['destination','orig'])

,orig_x,destination_x,orig_y,destination_y


In [16]:
df['Fraud--orig--destination']=df['Fraud']
df['Transfer--orig--destination']=1
create_facts_and_examples(df_= df, target='Fraud_chain--orig--destination', 
                          predicates=['Fraud--orig--destination', 'Transfer--orig--destination'], output_dir="examples/simple-transitivity-fraud-rule")

Fraud_chain--orig--destination 16042 16051 flag= True
Fraud_chain--orig--destination 16046 16003 flag= True
Fraud_chain--orig--destination 16077 16080 flag= False
Fraud_chain--orig--destination 16021 16032 flag= False
Fraud_chain--orig--destination 16010 16038 flag= False
Fraud_chain--orig--destination 16043 16019 flag= False
Fraud_chain--orig--destination 16058 16046 flag= False
Fraud_chain--orig--destination 16023 16042 flag= False
Fraud_chain--orig--destination 16059 16056 flag= False
Fraud_chain--orig--destination 16002 16060 flag= False
Fraud_chain--orig--destination 16062 16077 flag= False
Fraud_chain--orig--destination 16067 16024 flag= False
Fraud_chain--orig--destination 16082 16002 flag= False
Fraud_chain--orig--destination 16020 16013 flag= False
Fraud_chain--orig--destination 16081 16040 flag= False
Fraud_chain--orig--destination 16027 16019 flag= False
Fraud_chain--orig--destination 16041 16026 flag= False
Fraud_chain--orig--destination 16058 16066 flag= False
Fraud_chain-

In [17]:
df[['orig','destination','Fraud','Fraud_chain--orig--destination']] 

,orig,destination,Fraud,Fraud_chain--orig--destination
0,16042,16051,True,True
1,16077,16080,False,False
2,16021,16032,True,False
5,16010,16038,True,False
6,16043,16019,False,False
7,16058,16046,True,False
8,16023,16042,True,False
9,16059,16056,False,False
10,16002,16060,False,False
11,16062,16077,False,False


In [18]:
df[df['Fraud']==True]

,orig,destination,A,B,Fraud,Fraud_chain--orig--destination,From--orig,To--destination,Fraud--orig--destination,Transfer--orig--destination
0,16042,16051,False,True,True,True,1,1,True,1
2,16021,16032,True,True,True,False,1,1,True,1
5,16010,16038,True,True,True,False,1,1,True,1
7,16058,16046,True,True,True,False,1,1,True,1
8,16023,16042,True,True,True,False,1,1,True,1
13,16067,16024,True,True,True,False,1,1,True,1
15,16046,16003,False,True,True,True,1,1,True,1
19,16027,16019,True,True,True,False,1,1,True,1
21,16041,16026,True,True,True,False,1,1,True,1
22,16058,16066,True,True,True,False,1,1,True,1


In [19]:
create_facts_and_examples(df_= df, target='Fraud_chain--orig--destination', 
                          predicates=['Fraud--orig--destination'], output_dir="examples/fraud-and-fraud-chain")

Fraud_chain--orig--destination 16042 16051 flag= True
Fraud_chain--orig--destination 16046 16003 flag= True
Fraud_chain--orig--destination 16021 16032 flag= False
Fraud_chain--orig--destination 16010 16038 flag= False
Fraud_chain--orig--destination 16058 16046 flag= False
Fraud_chain--orig--destination 16023 16042 flag= False
Fraud_chain--orig--destination 16067 16024 flag= False
Fraud_chain--orig--destination 16027 16019 flag= False
Fraud_chain--orig--destination 16041 16026 flag= False
Fraud_chain--orig--destination 16058 16066 flag= False
Fraud_chain--orig--destination 16086 16014 flag= False
Fraud--orig--destination
Fraud--orig--destination 16042 16051 flag= True
Fraud--orig--destination 16021 16032 flag= True
Fraud--orig--destination 16010 16038 flag= True
Fraud--orig--destination 16058 16046 flag= True
Fraud--orig--destination 16023 16042 flag= True
Fraud--orig--destination 16067 16024 flag= True
Fraud--orig--destination 16046 16003 flag= True
Fraud--orig--destination 16027 16019